In [6]:
import urllib as ur
import json
import pandas as pd

In [7]:
def extract_Locations():
    #Formatando a request
    response = ur.request.urlopen('https://bookit.modo.coop/api/fleet').read().decode('UTF-8')
    json_Neighbourhood = json.loads(response)
    
    response = ur.request.urlopen('https://bookit.modo.coop/api/fleet/locations').read().decode('UTF-8')
    json_file = json.loads(response)
    
    NeighbourhoodsIDs = list(json_Neighbourhood['Neighbourhoods'].keys())

    LocationsIDs = []

    for i in NeighbourhoodsIDs:
        LocationsIDs.append(list(json_file['Neighbourhoods'][i]['Locations'].keys()))

    LocationsIDs = pd.DataFrame(LocationsIDs)

    LocationsIDs = LocationsIDs.T
    LocationsIDs.columns = NeighbourhoodsIDs

    #Colunas são os ids das vizinhaças e as linhas os estacionamentos
    return LocationsIDs

In [8]:
def extract_IDs():
    #Formatando a request
    response = ur.request.urlopen('https://bookit.modo.coop/api/fleet/locations').read().decode('UTF-8')
    json_file = json.loads(response)

    LocationsIDs = pd.DataFrame(extract_Locations())
    Cars = []
    auxCar = []
    IDs = []
    auxLoc = []

    for i in LocationsIDs.columns:
        for j in LocationsIDs[i]:
            if j == None:
                continue
            auxCar.append(list(json_file['Neighbourhoods'][i]['Locations'][j]['Cars']))
            #Percorre a lista com as IDs de localizações para pegar as multiplas ocorrências
            lista = list(json_file['Neighbourhoods'][i]['Locations'][j]['Cars'])
            for k in lista:
                auxLoc.append(j)
    
    #Reformatando para uma lista unidimensional
    for i in auxCar:
        for j in i:
            Cars.append(j)

    LocIDs = pd.DataFrame(auxLoc)
    CarIDs = pd.DataFrame(Cars) 
    IDs = pd.concat([LocIDs, CarIDs], axis=1)
    IDs.columns = ['LocationID', 'CarID']
    
    return IDs

In [9]:
import datetime as dt

auxTime = []
time = str(dt.datetime.now())

data = pd.DataFrame(extract_IDs())

for i in range(len(data)):
    auxTime.append(time)
    
auxTime = pd.DataFrame(auxTime)
    
data = pd.concat([data, auxTime], axis=1)
data.columns = ['LocationID', 'CarID', 'CaptureTime']

In [11]:
IDstr = ''
Start = []
End = []
Duration = []
FullyA = []
PartlyA = []
NotA = []
ReqStart = []
ReqEnd = []
ReqDuration = []

for i in range(len(data.CarID)):
    IDstr = ''
    if (i % 20 == 0):
        j = i
        #Separa as IDs em conjuntos de 20
        for k in range(j,i+20):
            #Evita indices inválidos
            try:
                if (k < i+19):
                    IDstr += str(data.CarID.iloc[k])+','
                else:
                    IDstr += str(data.CarID.iloc[k])
            except:
                print('Index out-of-bounds')
                break
        
        #Requisição de 20 IDs
        response = ur.request.urlopen('https://bookit.modo.coop/api/availability?cars='+IDstr+'').read().decode('UTF-8')
        json_Availability = json.loads(response)
        #Dados da requisição
        for i in range(20):
            ReqStart.append(json_Availability['Request']['RequestStart'])
            ReqEnd.append(json_Availability['Request']['RequestEnd'])
            ReqDuration.append(json_Availability['Request']['RequestDuration'])
        
        #Convertendo o dict dos IDs em uma string
        keys = list(json_Availability['Data'].keys())
        IDstr = IDstr.split(',')
        
        #Se ocorrer o erro de a ID não ter sido requisitada é completada com NaN
        for c in range(len(IDstr)):
            try:
                #Verifica se a ID foi ou não requisitada
                keys.index(IDstr[c])
                #Coletando dados
                Start.append(json_Availability['Data'][str(IDstr[c])]['StartTime'])
                End.append(json_Availability['Data'][str(IDstr[c])]['EndTime'])
                Duration.append(json_Availability['Data'][str(IDstr[c])]['Duration'])
                
                #Condições para disponibilidade
                if (json_Availability['Data'][str(IDstr[c])]['StartTime'] == False):
                    FullyA.append(0)
                    PartlyA.append(0)
                    NotA.append(1)
                #Para os carros totalmente disponíveis o tempo usado é o da ultima requisição
                elif (json_Availability['Data'][str(IDstr[c])]['Duration'] == str(ReqDuration[-1])):
                    FullyA.append(1)
                    PartlyA.append(0)
                    NotA.append(0)
                else:
                    FullyA.append(0)
                    PartlyA.append(1)
                    NotA.append(0)
                
            except:
                Start.append('NaN')
                End.append('NaN')
                FullyA.append(0)
                PartlyA.append(0)
                NotA.append(1)
            
Start = pd.DataFrame(Start)
End = pd.DataFrame(End)
Duration = pd.DataFrame(Duration)
FullyA = pd.DataFrame(FullyA)
PartlyA = pd.DataFrame(PartlyA)
NotA = pd.DataFrame(NotA)
ReqStart = pd.DataFrame(ReqStart)
ReqEnd = pd.DataFrame(ReqEnd)
ReqDuration = pd.DataFrame(ReqDuration)

data = pd.concat([data, FullyA, PartlyA, NotA, Start, End, Duration, ReqStart, ReqEnd, ReqDuration], axis=1)
data.columns = ['LocationID', 'CarID', 'CaptureTime', 'FullyAvailable', 'PartlyAvailable', 'NotAvailable',
                'StartTime', 'EndTime', 'Duration', 'RequestStart', 'RequestEnd', 'RequestDuration']


Index out-of-bounds


In [13]:
data.to_csv('ModoAPI_Data.csv', mode='a')